In [1]:
import os
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2, EfficientNetB0
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import numpy as np
from ultralytics import YOLO


Creating new Ultralytics Settings v0.0.6 file  
View Ultralytics Settings with 'yolo settings' or at 'C:\Users\JanTilles\AppData\Roaming\Ultralytics\settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [ ]:
# Load Pretrained Model
base_model = MobileNetV2(input_shape=(224, 224, 3), include_top=False, weights='imagenet')
base_model.trainable = False  # Freeze the base model initially

# Add task-specific layers with improved Dropout
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dropout(0.3), 
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5), 
    layers.Dense(4, activation='softmax')  # Adjust number of classes accordingly
])

# Implement Learning Rate Decay
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=0.001,
    decay_steps=1000,
    decay_rate=0.96,
    staircase=True
)

# Compile the model with improved learning rate tuning
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=lr_schedule),
    loss='sparse_categorical_crossentropy',  
    metrics=['accuracy']
)


In [9]:
AUTOTUNE = tf.data.AUTOTUNE

# Function to preprocess images
def preprocess_image(image, label):
    image = tf.image.resize(image, (224, 224)) / 255.0
    return image, label

# Load datasets using tf.data pipeline
train_ds = tf.keras.utils.image_dataset_from_directory(
    'dataset/train',
    image_size=(224, 224),
    batch_size=32
).map(preprocess_image).cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)

val_ds = tf.keras.utils.image_dataset_from_directory(
    'dataset/val',
    image_size=(224, 224),
    batch_size=32
).map(preprocess_image).cache().prefetch(buffer_size=AUTOTUNE)


Found 3919 files belonging to 4 classes.
Found 395 files belonging to 4 classes.


In [10]:
# Define callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
checkpoint_callback = ModelCheckpoint("best_model.keras", save_best_only=True, monitor="val_loss", verbose=1)

# Train the model
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=15,
    callbacks=[early_stopping, checkpoint_callback],
    verbose=1
)

# Fine-tune the entire model
base_model.trainable = True  # Unfreeze the base model

# Recompile model with a lower learning rate for fine-tuning
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
    loss='sparse_categorical_crossentropy',  # Or categorical if using one-hot
    metrics=['accuracy']
)

# Fine-tune the model with a new EarlyStopping instance
history_fine = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=15,
    callbacks=[early_stopping, checkpoint_callback],  # New callbacks
    verbose=1
)

Epoch 1/15
123/123 ━━━━━━━━━━━━━━━━━━━━ 0s 280ms/step - accuracy: 0.6310 - loss: 0.9443
Epoch 1: val_loss improved from inf to 0.47302, saving model to best_model.keras
123/123 ━━━━━━━━━━━━━━━━━━━━ 48s 324ms/step - accuracy: 0.6318 - loss: 0.9422 - val_accuracy: 0.8532 - val_loss: 0.4730
Epoch 2/15
123/123 ━━━━━━━━━━━━━━━━━━━━ 0s 276ms/step - accuracy: 0.8306 - loss: 0.4524
Epoch 2: val_loss improved from 0.47302 to 0.43870, saving model to best_model.keras
123/123 ━━━━━━━━━━━━━━━━━━━━ 38s 309ms/step - accuracy: 0.8307 - loss: 0.4524 - val_accuracy: 0.8608 - val_loss: 0.4387
Epoch 3/15
123/123 ━━━━━━━━━━━━━━━━━━━━ 0s 279ms/step - accuracy: 0.8504 - loss: 0.4045
Epoch 3: val_loss improved from 0.43870 to 0.42679, saving model to best_model.keras
123/123 ━━━━━━━━━━━━━━━━━━━━ 38s 312ms/step - accuracy: 0.8504 - loss: 0.4045 - val_accuracy: 0.8759 - val_loss: 0.4268
Epoch 4/15
123/123 ━━━━━━━━━━━━━━━━━━━━ 0s 293ms/step - accuracy: 0.8736 - loss: 0.3510
Epoch 4: val_loss did not improve fro

In [ ]:
# Load the YOLOv8 model
model = YOLO('yolov8n.pt')  # You can choose different versions like yolov8s.pt, yolov8m.pt, etc.

# Freeze the base model initially
for param in model.model.parameters():
    param.requires_grad = False

# Add task-specific layers (this part is more complex with YOLOv8 and typically involves modifying the model's head)
# YOLOv8 is not directly compatible with Keras layers, so this step is different from typical Keras models

# Implement Learning Rate Decay
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=0.001,
    decay_steps=1000,
    decay_rate=0.96,
    staircase=True
)

# Compile the model with improved learning rate tuning
# Note: YOLOv8 uses PyTorch, so you would use PyTorch optimizers and loss functions
optimizer = torch.optim.Adam(model.parameters(), lr=lr_schedule)
criterion = torch.nn.CrossEntropyLoss()

# Example training loop (simplified)
for epoch in range(num_epochs):
    for images, labels in dataloader:
        outputs = model(images)
        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()